In [1]:
# %pip install XlsxWriter

In [2]:
import xlsxwriter

In [3]:
# update here

nClientAuto30id = 2000

nClientInactive = 200
nClientActive = 1800

nClientLPEx = 600
nClientCNPEx = 100
nClientManualEx = 50
nClientCheck = 1050

nClientDataIssue = 150
nClientEgiRun = 900

nClientUnsuc = 200
nClientSuc = 700

nClientMedd = 300
meddLocked = 100
meddPass = 200

nClientCommercial = 400
commLocked = 150
commPass = 250

In [4]:
report_creation = {
    "first":{
        "Client from auto30id": nClientAuto30id
    },
    "second":{
        "Inactive":nClientInactive, 
        "Active": nClientActive
    },
    "third":{
        "Active LP Exclusions" : nClientLPEx , 
        "Active CNP Exclusions" : nClientCNPEx , 
        "Active Manual Exclusions" : nClientManualEx ,
        "Remaining for Data Quality Check": nClientCheck
    },
    "forth":{
        "Data Issues":nClientDataIssue, 
        "Eligible to Run": nClientEgiRun      
    },
    "fifth":{
        "Unsuccessful": nClientUnsuc,
        "Successful": nClientSuc
    },
    "sixth":{
        "Medd":[{'Medd': nClientMedd}, # first one is parent
                {"Locked":[{'meddLocked':meddLocked},{'brancha':40,'branchb':60}], # this line is 
                 "Passthrough":meddPass}], 
        "Commercial":[{"Commercial":nClientCommercial}, 
                      {"Locked":commLocked,"Passthrough":commPass}]
    },
    
    # add more levels based on future needs
    # the key names should be change to a more formal one
}

In [5]:
# create a workbook and a sheet
# change the path when needed

workbook = xlsxwriter.Workbook("MyExcel2.xlsx") # change the address of the workbook here
worksheet = workbook.add_worksheet()

In [6]:
parent_format = workbook.add_format({'bg_color': '#90EE90'})
# test_format = workbook.add_format({'bg_color': 'yellow'})

In [7]:
def add (levels, row, column): 
    '''
    The function 'add' can help write the N*2 data into the excel file based on the location of the starting cell (row,column).
    Starting cell: the top and left cell for each N*2 data
    
    levels: A dictionary. Have the name of the item and the value of it. 
    row: row index for the starting cell (row>=0)
    column: column index for the starting cell (col>=0)
    
    '''
    next_row = row 
    value_column = column + 1    
    
    for i in range(len(levels)):
        if isinstance(list(levels.values())[i],int):
            worksheet.write(next_row,column,list(levels.keys())[i])
            if i == len(levels)-1:
#                 worksheet.write(next_row,value_column,list(levels.values())[i],parent_format)
                worksheet.write(next_row,value_column,list(levels.values())[i])
            else:
                worksheet.write(next_row,value_column,list(levels.values())[i])              
            next_row += 1  ## move to the next row to insert a new record
        else:
            next_row = add(list(levels.values())[i][0],next_row,column)[0]
            next_row = add(list(levels.values())[i][1],next_row,column+1)[0]
                
          
    next_column = value_column # this becomes the next block's start column
    
    return next_row,next_column

In [8]:
current_row = 0
current_column = 0

In [9]:
for i in list(report_creation.keys()):
    if i == list(report_creation.keys())[-1]:
        current_row,current_column = add(report_creation[i],current_row,current_column)
    else:
        current_row,current_column = add(report_creation[i],current_row,current_column)
    
    print("finish",i)

finish first
finish second
finish third
finish forth
finish fifth
finish sixth


In [10]:
print(f"current row: {current_row},current_column:{current_column}",)

current row: 19,current_column:6


In [11]:
workbook.close()